## Offensive comment classifier

In [42]:
#Examine the data
#Can run Linux shell commands with !

!ls

TweetCommentClassifier.ipynb offenseval-trial.txt
offenseval-training-v1.tsv


In [45]:
#How many lines?
!wc -l offenseval-trial.txt

     320 offenseval-trial.txt


In [46]:
#What does the data look like?
!head -n 10 offenseval-trial.txt

In [5]:
#Looks like
#No header, 4 columns and tab seperated
#Lets read it in

In [6]:
#Import some key Python libraries for data analysis (Pandas and Numpy)
import pandas as pd
import numpy as np

In [7]:
#Import the data into a dataframe
df = pd.read_csv('offenseval-trial.txt', sep='\t', header=None)

In [8]:
#Look at the imported data
df.head()

,0,1,2,3
0,"@BreitbartNews OK Shannon, YOU tell the vetera...",NOT,NaN,NaN
1,@LeftyGlenn @jaredeker @BookUniverse @hashtagz...,NOT,NaN,NaN
2,Hot Mom Sucks Off Step Son In Shower 8 min htt...,OFF,UNT,NaN
3,bro these are some cute butt plugs I’m trying ...,OFF,UNT,NaN
4,Arizona Supreme Court strikes down state legis...,NOT,NaN,NaN


In [9]:
#For task (a) we are interested in columns 0 and 1, so let's drop 2 and 3
df = df.drop(columns = [2, 3])

In [10]:
df.head()

,0,1
0,"@BreitbartNews OK Shannon, YOU tell the vetera...",NOT
1,@LeftyGlenn @jaredeker @BookUniverse @hashtagz...,NOT
2,Hot Mom Sucks Off Step Son In Shower 8 min htt...,OFF
3,bro these are some cute butt plugs I’m trying ...,OFF
4,Arizona Supreme Court strikes down state legis...,NOT


In [11]:
#Let's understand the data a bit more
df.describe()

,0,1
count,320,320
unique,320,2
top,Interesting how guns get such a bad rap in the...,NOT
freq,1,243


From: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html

For object data (e.g. strings or timestamps), the result’s index will include count, unique, top, and freq. The top is the most common value. The freq is the most common value’s frequency. Timestamps also include the first and last items.

If multiple object values have the highest count, then the count and top results will be arbitrarily chosen from among those with the highest count.
        

In [12]:
#Name the columns- makes life easier
df.columns = ['tweet', 'label']

In [13]:
df.head()

,tweet,label
0,"@BreitbartNews OK Shannon, YOU tell the vetera...",NOT
1,@LeftyGlenn @jaredeker @BookUniverse @hashtagz...,NOT
2,Hot Mom Sucks Off Step Son In Shower 8 min htt...,OFF
3,bro these are some cute butt plugs I’m trying ...,OFF
4,Arizona Supreme Court strikes down state legis...,NOT


### Step 1: Preprocess the data, mainly data cleaning

In [22]:
#Split out the data, putting two two columns into two separate lists (so we don't cheat)
tweets = []
tweet_labels = []

In [23]:
tweets = df['tweet'].tolist()

In [24]:
tweet_labels = df['label'].tolist()

In [47]:
#Have a quick look at the lists
#tweets
#tweet_labels


## Step 2: Vectorise the data

We'll initialise a scikit-learn vector with the `CountVectorizer` class.  The data could be formatted in many different ways but as a start let's set it to lower case and exclude some common stop words.

The vectorizer will convert the data into vectors of features. As we'll use a `CountVector` the features are counts of the words that occur in the dataset.  We'll then fit the vectoriser onto our data and concert it to an array.

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
)

features = vectorizer.fit_transform(tweets)

features_nd = features.toarray() # for easy usage

Split the training data to get an evaluation set through Scikit-learn’s built-in cross_validation function. Provide the data and assign a training percentage (in this case, 80%).

In [30]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        tweet_labels,
        train_size=0.80, 
        random_state=1234)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Step 3: Build a Classifier model

There are many classifier models we could (and should) build and test.  We'll start with a simple model from Scikit-learn `LogisticRegression`.

In [32]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()

That's an empty model. Now let's train it to our specific dataset.  We'll use the scikit-learn `fit` method to get the ML classifier to *learn* the underlying functions that produce our results.

In [33]:
log_model = log_model.fit(X=X_train, y=y_train)

Use `log_model` to label our evaluation set we created earlier

In [36]:
y_pred = log_model.predict(X_test)

### How accurate is this?

In [39]:
#Choose a random set of tweets from our test data and then call the model on each.
import random
j = random.randint(0,len(X_test)-7)

for i in range(j,j+7):
    print(y_pred[0])
    ind = features_nd.tolist().index(X_test[i].tolist())
    print(tweets[ind].strip())

NOT
HAHAHA WHAT??? So does that mean the WWE Superstars that blocked me have to unblock me?? 😂😂 CAN I SUE??? #HolyShit https://t.co/SfS4l59U4X
NOT
As a football fan, F u c k this shit ⬇️ And it’s not even the first or last time this organization mishandles an important issue. 🙄 https://t.co/ZmEQSWr3Jz
NOT
@BreitbartNews It's hard to understand how representative Al Green wants to impeach a president. Who has brought black unemployment to its lowest levels. What hypocrisy we're all Americans and yet hatred and divisiveness is all the Democrat has to offer.
NOT
@BreitbartNews Jim Carey was APPROPRIATELY cast for the lead in DUMB and DUMBER.
NOT
@RickCarioti89 @AT12397451 @NYDailyNews You have called me “nuts” and the only reason I haven’t blocked you was because I thought you were a teenager and don’t know better. Now you insulted me again and you are done.
NOT
10 #Marvel and #DC Parodies That Are Not For Kids https://t.co/PEUycBcQOU https://t.co/UfObfezQl6
NOT
The Arizona Supreme Court 

Just looking at the data is a poor method to gauge accuracy and there are clearly some misclassifications.
Let's use scikit learn to calculate an accuracy score.

`sklearn.metrics.accuracy_score` will help us calculate what percentage of tweets are classified correctly.

In [41]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.84375


### **84%**

Not bad, better than random guessing but still far from perfect.

I've used a very simple classifier and a small dataset so let's go back and try again but this time with the full dataset.

In [50]:
#Step 0: Import the data into a dataframe
df = pd.read_csv('offenseval-training-v1.tsv', sep='\t')

In [51]:
df.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [53]:
df = df.drop(columns = ['id', 'subtask_b', 'subtask_c'])
df.head()

,tweet,subtask_a
0,@USER She should ask a few native Americans wh...,OFF
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,Amazon is investigating Chinese employees who ...,NOT
3,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,@USER @USER Obama wanted liberals &amp; illega...,NOT


In [55]:
#Step 1: Preprocess
#Split out the data, putting two two columns into two separate lists (so we don't cheat)
tweets = []
tweet_labels = []

tweets = df['tweet'].tolist()
tweet_labels = df['subtask_a'].tolist()

In [56]:
#Step 2: Vectorise
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
)

features = vectorizer.fit_transform(tweets)

features_nd = features.toarray() # for easy usage

In [57]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        tweet_labels,
        train_size=0.80, 
        random_state=1234)

In [58]:
#Step 3: Build a classifier
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()

In [59]:
#Train
log_model = log_model.fit(X=X_train, y=y_train)

In [61]:
#and label
y_pred = log_model.predict(X_test)

In [62]:
#Check accuracy

In [63]:
#Choose a random set of tweets from our test data and then call the model on each.
import random
j = random.randint(0,len(X_test)-7)

for i in range(j,j+7):
    print(y_pred[0])
    ind = features_nd.tolist().index(X_test[i].tolist())
    print(tweets[ind].strip())

NOT
@USER @USER Now Comes The Pain...............  #MAGA🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸  #WWG1WGA URL
NOT
@USER @USER @USER Chicago has been ran by Democrats for decades.  Why is it up to the GOP to solve the problems they created to begin with?  Act is absolutely code for gun control and that's not happening nor would it help anyway.  You need 1000 detectives.  Start hiring.
NOT
@USER Google got big by being the best search engine.  This is one reason why it is no longer a trustworthy search engine. @USER
NOT
@USER As it gets closer to November the left is in full on panic mode their talking points will forever be 25th Amendment. Ok idiots let’s see you try it.
NOT
@USER Spot on again Speedy. Always the victim when things aren’t going her way. Far cry from the Graf’s of the sport even though her stats will suggest she is one of the greats.
NOT
@USER HEY BEE ITCH YOU WILL BE INDICTED AND WILL GO TO GITMO
NOT
@USER LMBO! She's like Pelosi. She thinks she important because she is a female. She is a destrimen

In [65]:
#Calculate accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7530211480362538


### Less accurate!!!